In [1]:
import pandas as pd
import psycopg2
import os

from sqlalchemy import create_engine, text


In [2]:
host_db = 'ep-cool-frog-54980598.eu-central-1.aws.neon.tech'
user_db = ''
pswd_db = ''
name_db = 'clients_streamlit'
print(host_db, user_db, pswd_db, name_db)

conn_string = f'postgresql://{user_db}:{pswd_db}@{host_db}/{name_db}?sslmode=require'
db = create_engine(conn_string)
conn_eng = db.connect()

ep-cool-frog-54980598.eu-central-1.aws.neon.tech NGeraskina x8M6PfJDBLCc clients_streamlit


In [3]:
# присоединение всех справочников
conn_eng = db.connect()
df_clients = pd.read_sql_query(text('SELECT * FROM clients'), con = conn_eng).rename(columns = {"ID":"ID_CLIENT"})
df_clients['status_pens'] = df_clients['SOCSTATUS_PENS_FL'].map({1:'пенсионер', 0:'не пенсионер'})
df_clients['status_work'] = df_clients['SOCSTATUS_WORK_FL'].map({1:'работает', 0:'не работает'})

for i in ['job', 'salary', 'last_credit']:
  df_spr = pd.read_sql_query(text(f'SELECT * FROM {i}'), con = conn_eng)
  df_clients = df_clients.merge(df_spr, how = 'left', left_on = 'ID_CLIENT', right_on = 'ID_CLIENT')

df_clients

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,status_work,GEN_INDUSTRY,GEN_TITLE,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,...,работает,Другие сферы,Работник сферы услуг,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,5588.0,6.0,1000.0
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,...,работает,Торговля,Специалист,Участие в основ. деятельности,5.0,от 10000 до 20000 руб.,10000.0,19498.0,12.0,0.0
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,...,работает,Другие сферы,Руководитель высшего звена,Участие в основ. деятельности,360.0,от 20000 до 50000 руб.,30000.0,15470.0,3.0,15000.0
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,...,работает,Государственная служба,Специалист,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,13960.0,6.0,2500.0
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,...,работает,Другие сферы,Специалист,Участие в основ. деятельности,12.0,от 10000 до 20000 руб.,15000.0,11890.0,6.0,8000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16295,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,...,работает,Сборочные производства,Специалист,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,8000.0,11750.0,4.0,1900.0
16296,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,...,работает,Торговля,Специалист,Участие в основ. деятельности,24.0,от 20000 до 50000 руб.,12000.0,12350.0,6.0,1380.0
16297,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,...,работает,Торговля,Специалист,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,9000.0,4915.0,10.0,2000.0
16298,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,...,работает,Транспорт,Специалист,Участие в основ. деятельности,36.0,от 10000 до 20000 руб.,12000.0,5860.0,3.0,2000.0


In [4]:
# построение всех агрегатов для присоединения инфы

conn_eng = db.connect()
df_loan = pd.read_sql_query(text(f'SELECT * FROM loan'), con = conn_eng)
df_close_loan = pd.read_sql_query(text(f'SELECT * FROM close_loans'), con = conn_eng)
df_loan = df_loan.merge(df_close_loan, how = 'left', on = 'ID_LOAN')
df_loan_new = df_loan.groupby('ID_CLIENT').sum().reset_index().drop(columns = ['ID_LOAN']).rename(columns = {'CLOSED_FL': 'CLOSED_LOANS'})
df_loan_new = df_loan_new.merge(df_loan.groupby('ID_CLIENT').count().reset_index().rename(columns = {'ID_LOAN':"CNT_LOANS"})[['ID_CLIENT', 'CNT_LOANS']], on = 'ID_CLIENT')
df_loan_new

,ID_CLIENT,CLOSED_LOANS,CNT_LOANS
0,106804370,1,1
1,106804371,1,1
2,106804372,1,2
3,106804373,1,1
4,106804374,1,2
...,...,...,...
15218,106819588,2,3
15219,106819589,0,1
15220,106819590,0,1
15221,106819591,0,1


In [5]:
conn_eng = db.connect()
df_target = pd.read_sql_query(text(f'SELECT * FROM target'), con = conn_eng)
df_target = df_target.groupby('ID_CLIENT').sum().reset_index().drop(columns = ['AGREEMENT_RK']).rename(columns = {'TARGET': 'CNT_TARGET'})
df_target

,ID_CLIENT,CNT_TARGET
0,106804370,0
1,106804371,0
2,106804372,0
3,106804373,0
4,106804374,0
...,...,...
15218,106819588,0
15219,106819589,0
15220,106819590,0
15221,106819591,0


In [6]:
df_clients = df_clients.merge(df_target, how = 'left', on = 'ID_CLIENT').merge(df_loan_new, how = 'left', on = 'ID_CLIENT')
df_clients

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,...,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,5588.0,6.0,1000.0,1.0,0.0,1.0
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,...,Участие в основ. деятельности,5.0,от 10000 до 20000 руб.,10000.0,19498.0,12.0,0.0,0.0,0.0,1.0
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,...,Участие в основ. деятельности,360.0,от 20000 до 50000 руб.,30000.0,15470.0,3.0,15000.0,0.0,1.0,1.0
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,...,Участие в основ. деятельности,3.0,от 20000 до 50000 руб.,25000.0,13960.0,6.0,2500.0,0.0,0.0,1.0
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,...,Участие в основ. деятельности,12.0,от 10000 до 20000 руб.,15000.0,11890.0,6.0,8000.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16295,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,...,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,8000.0,11750.0,4.0,1900.0,0.0,2.0,2.0
16296,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,...,Участие в основ. деятельности,24.0,от 20000 до 50000 руб.,12000.0,12350.0,6.0,1380.0,0.0,0.0,1.0
16297,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,...,Участие в основ. деятельности,36.0,от 5000 до 10000 руб.,9000.0,4915.0,10.0,2000.0,0.0,0.0,1.0
16298,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,...,Участие в основ. деятельности,36.0,от 10000 до 20000 руб.,12000.0,5860.0,3.0,2000.0,0.0,1.0,1.0


In [7]:
# удалю ненужные столбцы: внешние айди других витрин, неинформативные столбцы

df_clients = df_clients.drop(columns = ['SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL'])

In [8]:
# "раскодирую" данные

df_clients['GENDER'] = df_clients['GENDER'].map({1:'Мужчина', 0:"Женщина"})
df_clients['FL_PRESENCE_FL'] = df_clients['FL_PRESENCE_FL'].map({1:'Есть', 0:"Нет"})

In [9]:
# исследую на корректность и дублирование

df_clients = df_clients.drop_duplicates()
df_clients.shape

(16000, 26)

In [10]:
# проверю нет ли тех у кого первый платеж больше суммы кредита

df_clients[df_clients.FST_PAYMENT > df_clients.CREDIT]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS
29,106804376,35,Женщина,Высшее,Разведен(а),1,1,Вологодская область,Вологодская область,Вологодская область,...,Участие в основ. деятельности,6.0,от 20000 до 50000 руб.,15000.0,10450.0,3.0,15000.0,0.0,2.0,2.0
34,106807419,56,Мужчина,Неполное среднее,Состою в браке,3,0,Пермская область,Пермская область,Пермская область,...,Участие в основ. деятельности,90.0,от 20000 до 50000 руб.,20000.0,11400.0,4.0,15000.0,0.0,1.0,1.0
40,106815597,36,Женщина,Среднее,Состою в браке,1,1,Костромская область,Костромская область,Костромская область,...,Участие в основ. деятельности,48.0,от 20000 до 50000 руб.,15000.0,11550.0,6.0,14760.0,0.0,2.0,2.0
99,106817975,63,Мужчина,Среднее,Разведен(а),2,0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,...,None,NaN,от 5000 до 10000 руб.,6000.0,8990.0,3.0,10000.0,0.0,1.0,1.0
102,106805994,38,Женщина,Высшее,Состою в браке,1,1,Карелия,Карелия,Карелия,...,Участие в основ. деятельности,24.0,от 20000 до 50000 руб.,16000.0,10000.0,3.0,22000.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16182,106818316,38,Мужчина,Высшее,Не состоял в браке,1,1,Ростовская область,Ростовская область,Ростовская область,...,Участие в основ. деятельности,60.0,от 10000 до 20000 руб.,10000.0,8550.0,6.0,9000.0,0.0,1.0,2.0
16188,106816135,35,Мужчина,Высшее,Разведен(а),1,1,Владимирская область,Владимирская область,Владимирская область,...,Участие в основ. деятельности,60.0,от 10000 до 20000 руб.,12000.0,8490.0,3.0,9000.0,0.0,1.0,1.0
16263,106806965,57,Мужчина,Среднее специальное,Состою в браке,0,0,Кировская область,Кировская область,Кировская область,...,None,NaN,от 10000 до 20000 руб.,4900.0,3640.0,3.0,7000.0,0.0,1.0,1.0
16291,106805881,31,Мужчина,Среднее специальное,Состою в браке,1,1,Краснодарский край,Краснодарский край,Краснодарский край,...,Участие в основ. деятельности,60.0,от 10000 до 20000 руб.,9000.0,3119.0,3.0,7500.0,0.0,1.0,1.0


In [11]:
# удалю этих чудиков
df_clients = df_clients[df_clients.FST_PAYMENT <= df_clients.CREDIT]

In [12]:
# проверю на ошибки по типу отрицательных сроков кредитов и отрицательных платежей/сумм

df_clients[(df_clients.FST_PAYMENT < 0) | (df_clients.PERSONAL_INCOME < 0) | (df_clients.WORK_TIME < 0)]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS


In [13]:
df_clients[(df_clients.CREDIT <= 0) | (df_clients.TERM <= 0)]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS


In [14]:
df_clients[df_clients.PERSONAL_INCOME<1000]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS
3480,106807861,28,Женщина,Среднее,Состою в браке,1,1,Читинская область,Читинская область,Читинская область,...,Участие в основ. деятельности,45.0,от 20000 до 50000 руб.,24.0,63306.0,18.0,7034.0,0.0,0.0,1.0


In [15]:
df_clients['PERSONAL_INCOME'] = df_clients['PERSONAL_INCOME'].apply(lambda x: x*1000 if x<1000 else x)
df_clients[df_clients.PERSONAL_INCOME<1000]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS


In [16]:
# проверю выбросы во времени работы на текущем месте: человек не может работать больше, чем (возраст-15 лет)*365
df_clients[df_clients.WORK_TIME>((df_clients.AGE - 15)*365)]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS
3976,106813354,25,Мужчина,Среднее специальное,Не состоял в браке,0,0,Коми,Коми,Коми,...,Участие в основ. деятельности,2867959.0,от 20000 до 50000 руб.,9000.0,19622.0,6.0,5000.0,0.0,1.0,1.0


In [17]:
df_clients = df_clients[df_clients.WORK_TIME<((df_clients.AGE - 15)*365)]

In [19]:
# проверю есть ли люди, у который закрытых кредитов больше чем кол-во кредитов
df_clients[df_clients.CNT_LOANS < df_clients.CLOSED_LOANS]

,ID_CLIENT,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,...,JOB_DIR,WORK_TIME,FAMILY_INCOME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,CNT_TARGET,CLOSED_LOANS,CNT_LOANS


In [18]:
df_clients.to_csv('clients.csv')